In [ ]:
import seagul.envs
import ray
import ray.rllib.agents.ars as ars
from ray.tune.logger import pretty_print
import tensorflow as tf
import time
from tqdm import trange
from ray import tune

from ray.tune.registry import register_env
from seagul.envs.mujoco.five_link import FiveLinkWalkerEnv

env_name = "Walker2d-v3"

###
def env_creator(env_config):
    return FiveLinkWalkerEnv()  # return an env instance

register_env("five_link-v3", env_creator)
###

ray.init()
config = ars.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 8
config["eager"] = False
config["model"]["fcnet_hiddens"] = []

trainer = ars.ARSTrainer(config=config, env=env_name)
# Can optionally call trainer.restore(path) to load a checkpoint.
for i in trange(5000):
   # Perform one iteration of training the policy with ARS
    result = trainer.train()


    
checkpoint = trainer.save()
print("checkpoint saved at", checkpoint)
print(pretty_print(result))
ray.shutdown()

/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sgillen/miniconda3/envs/ray/lib/python3.6/s

(pid=30706) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=30706)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=30706) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=30706)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=30706) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=30706)   _np_qint16 = np.dtype([("q

  0%|          | 1/5000 [00:04<6:14:59,  4.50s/it]/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/ray/rllib/agents/ars/ars.py:289: RuntimeWarning: Mean of empty slice.
  episode_len_mean=eval_lengths.mean(),
/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  0%|          | 2/5000 [00:04<4:33:03,  3.28s/it]/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/ray/rllib/agents/ars/ars.py:289: RuntimeWarning: Mean of empty slice.
  episode_len_mean=eval_lengths.mean(),
/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  0%|          | 4/5000 [00:05<2:32:42,  1.83s/it]/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/ray/rllib/agents/ars/ars.py:289: RuntimeWarning: Mean of empty slice.


In [ ]:
import numpy as np
trainer.policy.compute(np.random.randn(13))



In [ ]:
import seagul.envs
import ray
import ray.rllib.agents.ars as ars
from ray.tune.logger import pretty_print
import tensorflow as tf


from ray.tune.registry import register_env
from seagul.envs.mujoco.five_link import FiveLinkWalkerEnv

def env_creator(env_config):
    return FiveLinkWalkerEnv()  # return an env instance

register_env("five_link-v3", env_creator)

ray.init()
config = ars.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
config["eager"] = False
trainer = ars.ARSTrainer(config=config, env="five_link-v3")

trainer.load("/Users/sgillen/ray_results/ARS_five_link-v3_2019-10-21_14-02-55lwgccrp3/checkpoint_10/checkpoint-10") 
trainer.restore("/Users/sgillen/ray_results/ARS_five_link-v3_2019-10-21_14-02-55lwgccrp3/checkpoint_10/checkpoint-10")


In [6]:
import gym
import numpy as np
import matplotlib.pyplot as plt

env = gym.make(env_name)
obs = env.reset()

action_hist = []
state_hist  = []
reward_hist = []

done = False
while not done:
    actions = trainer.compute_action(obs.flatten())
    obs, reward, done, _ = env.step(np.asarray(actions))
    action_hist.append(np.copy(np.clip(actions, -10, 10)))
    state_hist.append(np.copy(obs))
    reward_hist.append(np.copy(reward))
    if done:
        break


print(sum(reward_hist))
plt.plot(action_hist)
plt.figure()
plt.plot(state_hist)

NameError: name 'env_name' is not defined